In [1]:
import pandas as pd
import time
import torch
import random
import torch

In [2]:
def encrypt_sequence(x):
    y = []
    y.append(x[0])
    for i in range(1, len(x)):
        if x[i] + x[0] >= 10:
            y.append(x[i] + x[0] - 10)
        else:
            y.append(x[i] + x[0])
    return y

In [3]:
lst = []
num_chars = 12
for i in range(num_chars):
    lst.append(random.randint(0, 9))
print(lst)
lst_encrypted = encrypt_sequence(lst)
print(encrypt_sequence(lst))

[1, 3, 6, 4, 6, 5, 0, 4, 7, 0, 3, 5]
[1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6]


In [4]:
def dataset(num_examples):

    dataset_in = []
    dataset_out = []
    for x in range(num_examples):
        seq_out = []
        for i in range(num_chars):
            seq_out.append(random.randint(0, 9))
        seq_in = encrypt_sequence(seq_out)
        
        dataset_in.append(torch.tensor(seq_out))
        dataset_out.append(torch.tensor(seq_in))
        
    return torch.stack(dataset_in), torch.stack(dataset_out)

In [5]:
num_examples = 5000
di, do = dataset(num_examples)
di.shape, do.shape

(torch.Size([5000, 12]), torch.Size([5000, 12]))

In [6]:
class NN(torch.nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = torch.nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first = True)
        self.output = torch.nn.Linear(num_hiddens, num_classes)
    
    def forward(self, X):
        out = self.embedding(X)
        o1, state = self.hidden(out)
        predictions = self.output(o1)
        return predictions

In [7]:
test_dsi, test_dso = dataset(100)
test_dsi.shape, test_dso.shape

(torch.Size([100, 12]), torch.Size([100, 12]))

## 1. GRU

In [24]:
model = NN(torch.nn.GRU, 10, 128, 256, 10)

In [25]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [26]:
for ep in range(25):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(di) / 50)):
        batch = di[i * 10:(i + 1) * 10]
        batch2 = do[i * 10:(i + 1) * 10]
        
        optimizer.zero_grad()
        answers = model.forward(batch)
        #print(answers.shape)
        loss = criterion(answers.view(-1, 10), batch2.flatten())
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if True:
        acc = 0
        for i in range(len(test_dsi)):
            o2 = model(torch.tensor(test_dsi[i]))
            #print(o2.shape)
            w = torch.argmax(o2, dim = 1)
            #print(w)
            o1 = torch.tensor(encrypt_sequence(test_dsi[i]))
            #print(o1)
            c = 0
            for j in range(num_chars):
                if w[j] == o1[j]:
                    c += 1
            acc += c/num_chars
        acc /= len(test_dsi)
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}, Accuracy: {:.3f}".format(ep, time.time() - start, train_loss / train_passed, acc))


<ipython-input-26-2028a5faea00>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o2 = model(torch.tensor(test_dsi[i]))


Epoch 0. Time: 1.008, Train loss: 2.296, Accuracy: 0.153
Epoch 1. Time: 1.012, Train loss: 2.272, Accuracy: 0.158
Epoch 2. Time: 1.028, Train loss: 2.263, Accuracy: 0.157
Epoch 3. Time: 1.062, Train loss: 2.255, Accuracy: 0.157
Epoch 4. Time: 1.031, Train loss: 2.246, Accuracy: 0.158
Epoch 5. Time: 1.018, Train loss: 2.234, Accuracy: 0.159
Epoch 6. Time: 0.979, Train loss: 2.214, Accuracy: 0.166
Epoch 7. Time: 1.044, Train loss: 2.186, Accuracy: 0.177
Epoch 8. Time: 1.015, Train loss: 2.148, Accuracy: 0.199
Epoch 9. Time: 1.023, Train loss: 2.085, Accuracy: 0.253
Epoch 10. Time: 1.014, Train loss: 1.964, Accuracy: 0.285
Epoch 11. Time: 1.034, Train loss: 1.868, Accuracy: 0.350
Epoch 12. Time: 1.007, Train loss: 1.765, Accuracy: 0.450
Epoch 13. Time: 1.031, Train loss: 1.629, Accuracy: 0.567
Epoch 14. Time: 0.918, Train loss: 1.461, Accuracy: 0.676
Epoch 15. Time: 0.903, Train loss: 1.279, Accuracy: 0.784
Epoch 16. Time: 0.894, Train loss: 1.089, Accuracy: 0.881
Epoch 17. Time: 0.967, T

In [27]:
print(encrypt_sequence(lst))
print(torch.argmax(model(torch.tensor(lst)), dim = 1))

[1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6]
tensor([1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6])


## 2. RNN

In [32]:
model2 = NN(torch.nn.RNN, 10, 128, 256, 10)

In [33]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr = 0.0001)

In [34]:
for ep in range(70):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(di) / 50)):
        batch = di[i * 10:(i + 1) * 10]
        batch2 = do[i * 10:(i + 1) * 10]
        
        optimizer.zero_grad()
        answers = model2.forward(batch)
        #print(answers.shape)
        loss = criterion(answers.view(-1, 10), batch2.flatten())
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if True:
        acc = 0
        for i in range(len(test_dsi)):
            o2 = model2(torch.tensor(test_dsi[i]))
            #print(o2.shape)
            w = torch.argmax(o2, dim = 1)
            #print(w)
            o1 = torch.tensor(encrypt_sequence(test_dsi[i]))
            #print(o1)
            c = 0
            for j in range(num_chars):
                if w[j] == o1[j]:
                    c += 1
            acc += c/num_chars
        acc /= len(test_dsi)
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}, Accuracy: {:.3f}".format(ep, time.time() - start, train_loss / train_passed, acc))


<ipython-input-34-a728668cc391>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o2 = model2(torch.tensor(test_dsi[i]))


Epoch 0. Time: 0.519, Train loss: 2.296, Accuracy: 0.157
Epoch 1. Time: 0.497, Train loss: 2.267, Accuracy: 0.157
Epoch 2. Time: 0.500, Train loss: 2.262, Accuracy: 0.157
Epoch 3. Time: 0.463, Train loss: 2.259, Accuracy: 0.157
Epoch 4. Time: 0.476, Train loss: 2.256, Accuracy: 0.157
Epoch 5. Time: 0.456, Train loss: 2.253, Accuracy: 0.157
Epoch 6. Time: 0.557, Train loss: 2.250, Accuracy: 0.157
Epoch 7. Time: 0.509, Train loss: 2.246, Accuracy: 0.157
Epoch 8. Time: 0.532, Train loss: 2.241, Accuracy: 0.157
Epoch 9. Time: 0.509, Train loss: 2.235, Accuracy: 0.158
Epoch 10. Time: 0.506, Train loss: 2.226, Accuracy: 0.158
Epoch 11. Time: 0.489, Train loss: 2.211, Accuracy: 0.159
Epoch 12. Time: 0.454, Train loss: 2.179, Accuracy: 0.166
Epoch 13. Time: 0.471, Train loss: 2.136, Accuracy: 0.178
Epoch 14. Time: 0.483, Train loss: 2.093, Accuracy: 0.203
Epoch 15. Time: 0.458, Train loss: 2.052, Accuracy: 0.225
Epoch 16. Time: 0.515, Train loss: 2.016, Accuracy: 0.235
Epoch 17. Time: 0.470, T

In [35]:
print(encrypt_sequence(lst))
print(torch.argmax(model2(torch.tensor(lst)), dim = 1))

[1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6]
tensor([1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6])


## 3. LSTM

In [42]:
model3 = NN(torch.nn.LSTM, 10, 128, 256, 10)

In [43]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr = 0.0001)

In [44]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(di) / 50)):
        batch = di[i * 10:(i + 1) * 10]
        batch2 = do[i * 10:(i + 1) * 10]
        
        optimizer.zero_grad()
        answers = model3.forward(batch)
        #print(answers.shape)
        loss = criterion(answers.view(-1, 10), batch2.flatten())
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if True:
        acc = 0
        for i in range(len(test_dsi)):
            o2 = model3(torch.tensor(test_dsi[i]))
            #print(o2.shape)
            w = torch.argmax(o2, dim = 1)
            #print(w)
            o1 = torch.tensor(encrypt_sequence(test_dsi[i]))
            #print(o1)
            c = 0
            for j in range(num_chars):
                if w[j] == o1[j]:
                    c += 1
            acc += c/num_chars
        acc /= len(test_dsi)
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}, Accuracy: {:.3f}".format(ep, time.time() - start, train_loss / train_passed, acc))


<ipython-input-44-e8d1c7323ce7>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o2 = model3(torch.tensor(test_dsi[i]))


Epoch 0. Time: 1.844, Train loss: 2.293, Accuracy: 0.158
Epoch 1. Time: 1.767, Train loss: 2.276, Accuracy: 0.158
Epoch 2. Time: 1.783, Train loss: 2.265, Accuracy: 0.157
Epoch 3. Time: 1.913, Train loss: 2.255, Accuracy: 0.157
Epoch 4. Time: 1.715, Train loss: 2.240, Accuracy: 0.161
Epoch 5. Time: 1.829, Train loss: 2.206, Accuracy: 0.187
Epoch 6. Time: 1.782, Train loss: 2.116, Accuracy: 0.281
Epoch 7. Time: 1.755, Train loss: 1.979, Accuracy: 0.350
Epoch 8. Time: 1.839, Train loss: 1.815, Accuracy: 0.444
Epoch 9. Time: 1.871, Train loss: 1.607, Accuracy: 0.570
Epoch 10. Time: 1.986, Train loss: 1.368, Accuracy: 0.704
Epoch 11. Time: 1.974, Train loss: 1.117, Accuracy: 0.822
Epoch 12. Time: 2.047, Train loss: 0.878, Accuracy: 0.932
Epoch 13. Time: 2.015, Train loss: 0.679, Accuracy: 0.976
Epoch 14. Time: 2.038, Train loss: 0.519, Accuracy: 0.990
Epoch 15. Time: 2.006, Train loss: 0.397, Accuracy: 0.994
Epoch 16. Time: 2.016, Train loss: 0.305, Accuracy: 0.997
Epoch 17. Time: 2.058, T

In [45]:
print(encrypt_sequence(lst))
print(torch.argmax(model2(torch.tensor(lst)), dim = 1))

[1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6]
tensor([1, 4, 7, 5, 7, 6, 1, 5, 8, 1, 4, 6])
